## Генерация текста с помощью RNN

В этом ноутбуке сначала необходимо:
1. Разобрать пример с гугла по ссылке (https://www.tensorflow.org/tutorials/text/text_generation?hl=ru)
1. Постараться преобразовать пример в рабочую модель для генерации daily analysis статей

In [3]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [13]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# Take a look at the first 250 characters in text
print(text[:250])
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

Length of text: 1115394 characters
65 unique characters


## Обработка текста

In [9]:
example_texts = ['фыва', 'олдж']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars.numpy()

array([[b'\xd1\x84', b'\xd1\x8b', b'\xd0\xb2', b'\xd0\xb0'],
       [b'\xd0\xbe', b'\xd0\xbb', b'\xd0\xb4', b'\xd0\xb6']], dtype=object)

In [10]:
# Преобразование символов в векторную строку
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[1, 1, 1, 1], [1, 1, 1, 1]]>

In [11]:
ids_from_chars.get_vocabulary()

['',
 '[UNK]',
 '\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [15]:
# Обратное преобразование
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)
chars = chars_from_ids(ids)
print(chars)

<tf.RaggedTensor [[b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]'], [b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]']]>


In [16]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## Задача прогнозирования

Для данного символа или последовательности символов какой следующий символ наиболее вероятен? Это задача, которой вы обучаете модель. Входными данными для модели будет последовательность символов, и вы обучаете модель предсказывать выходные данные - следующий символ на каждом временном шаге.

## Создавайте обучающие примеры и цели

Затем разделите текст на примеры последовательностей. Каждая входная последовательность будет содержать символы seq_length из текста.

Для каждой входной последовательности соответствующие целевые объекты содержат текст одинаковой длины, за исключением смещения на один символ вправо.

In [17]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([20, 49, 58, ..., 47, 10,  2], dtype=int64)>

In [18]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [19]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [20]:
# Объединяем обратно

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy().decode('utf-8'))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be d
one: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citi


In [21]:
# Вот функция, которая принимает последовательность как ввод, дублирует и сдвигает ее, 
# чтобы выровнять ввод и метку для каждого временного шага
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy().decode('utf-8'))
    print("Target:", text_from_ids(target_example).numpy().decode('utf-8'))

Input : First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
Target: irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


## Создавайте тренировочные партии

Вы использовали tf.data чтобы разбить текст на управляемые последовательности. Но прежде чем вводить эти данные в модель, вам необходимо перетасовать данные и упаковать их в пакеты.

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Постройте модель

В этом разделе модель определяется как подкласс keras.Model (подробнее см. Создание новых слоев и моделей через создание подклассов ).

Эта модель состоит из трех слоев:

* tf.keras.layers.Embedding : входной слой. Обучаемая таблица поиска, которая будет отображать каждый идентификатор символа в вектор с размерами embedding_dim ;
* tf.keras.layers.GRU : тип RNN с размером units=rnn_units (здесь также можно использовать слой LSTM.)
* tf.keras.layers.Dense : выходной слой с выходными значениями vocab_size . Для каждого символа в словаре используется один логит. Это логарифмическая вероятность каждого символа в соответствии с моделью.

In [24]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
print(vocab_size)
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

65


In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [26]:
print(len(ids_from_chars.get_vocabulary()))
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

67


## Попробуйте модель

Теперь запустите модель, чтобы убедиться, что она ведет себя должным образом.

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    
model.summary()

(64, 100, 67) # (batch_size, sequence_length, vocab_size)
Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  17152     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  68675     
Total params: 4,024,131
Trainable params: 4,024,131
Non-trainable params: 0
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

array([34, 26, 11, 50,  4, 20,  4, 31, 46, 44, 28, 21, 48, 57,  0, 26, 47,
       11, 40, 32,  6, 61, 48, 66, 41, 30, 30, 56, 36, 57, 29, 27, 54,  5,
       23, 32, 64, 64, 20, 11, 66, 40, 30, 40, 63, 36, 13, 59, 22, 28, 65,
       49, 31, 27, 16,  3, 24, 22, 53, 60, 55, 22, 10, 53, 60, 49,  7,  7,
       10, 31, 51, 27, 28, 41, 47, 60, 63, 36,  2,  9, 13, 60, 54, 63, 23,
       23, 64, 20, 52, 60,  9, 62,  9, 39, 19, 45,  2, 12,  3, 60],
      dtype=int64)

In [29]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' sufficient to serve it?\n\nELBOW:\nFaith, sir, few of any wit in such matters: as they\nare chosen, the'

Next Char Predictions:
 b"TL3j!F!QfdNGhqLg3ZR&uhzaPPpVqOMn$IRxxF3zZPZwV;sHNyiQMB JHmtoH.mti''.QkMNagtwV\n-;tnwIIxFlt-v-YEe\n: t"


## Обучите модель

На этом этапе проблему можно рассматривать как стандартную задачу классификации. Учитывая предыдущее состояние RNN и ввод этого временного шага, спрогнозируйте класс следующего символа.

In [30]:
# Прикрепите оптимизатор и функцию потерь
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)
tf.exp(mean_loss).numpy()

Prediction shape:  (64, 100, 67)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.205507


67.05457

In [31]:
# Настройте процедуру обучения с tf.keras.Model.compile метода tf.keras.Model.compile . 
# Используйте tf.keras.optimizers.Adam с аргументами по умолчанию и функцией потерь.

model.compile(optimizer='adam', loss=loss)

In [32]:
# Используйте tf.keras.callbacks.ModelCheckpoint чтобы убедиться, что контрольные точки сохраняются во время обучения:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
# Провести обучение
EPOCHS = 5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
  4/172 [..............................] - ETA: 4:04 - loss: 4.1101

KeyboardInterrupt: 

In [59]:
# Создать текст

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [60]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [64]:
start = time.time()
states = None
next_char = tf.constant(['OXFORD:'])
result = [next_char]

for n in range(500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

OXFORD:
Yet withdefore flood, and call'd on. Not give them, my poor
Mansal humbland aw, let much made
your sintly and him, my dost to son,
And be bone in door age.

BRUSTOND:
Why Nore.

ANGELO:
Diedesly for it. Assister or clomem, they
Ware, for husband; for your hang-to hear me so speak.

DUCHESS OF YORK:
Marry she to denight the morning high of
When ever his montunes bount your old with suchis name to do wet you
honey on husband, and with such flumber, and love to Fray your wrench
That give fible cur 

________________________________________________________________________________

Run time: 0.8199756145477295


In [88]:
start = time.time()
states = None
next_char = tf.constant(['apple'])
result = [next_char]

for n in range(200):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result, '\n\n' + '_'*80)


print(f"\nRun time: {end - start}")

tf.Tensor([b"apple: he would de\nyours. Bry Velint:\nI you strike him here: they bryalking them in\nAbove enemy, hear me not. Say this yep unto my streed?\nNot full, mine encive, I should have die to those\nAnd 'Be that, no"], shape=(1,), dtype=string) 

________________________________________________________________________________

Run time: 0.33849358558654785


## Генерация аналитики

In [4]:
path_to_analfile = 'gen_text.csv'
an_text = open(path_to_analfile, 'rb').read().decode(encoding='utf-8')
an_vocab = sorted(set(an_text))
print('{} unique characters'.format(len(an_vocab)))
an_text[:100]

81 unique characters


'ТРЕНД ВВЕРХ;Изменение цены за последние полгода составляет 26.39%, а за последний месяц 11.72%. Напр'

In [5]:
# Примеры и проверка работы с текстом и преобразованиями
example_texts = [an_text[:10], an_text[11:20]]

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
print(chars.numpy())

# Преобразование символов в векторную строку
ids_from_chars = preprocessing.StringLookup(vocabulary=list(an_vocab))
ids = ids_from_chars(chars)
print(ids)

# Обратное преобразование
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)
chars = chars_from_ids(ids)
print(chars.numpy())
print(tf.strings.reduce_join(chars, axis=-1))

[array([b'\xd0\xa2', b'\xd0\xa0', b'\xd0\x95', b'\xd0\x9d', b'\xd0\x94',
       b' ', b'\xd0\x92', b'\xd0\x92', b'\xd0\x95', b'\xd0\xa0'],
      dtype=object)
 array([b';', b'\xd0\x98', b'\xd0\xb7', b'\xd0\xbc', b'\xd0\xb5',
       b'\xd0\xbd', b'\xd0\xb5', b'\xd0\xbd', b'\xd0\xb8'], dtype=object)]
<tf.RaggedTensor [[45, 43, 36, 40, 35, 4, 33, 33, 36, 43], [19, 38, 59, 64, 57, 65, 57, 65, 60]]>
[array([b'\xd0\xa2', b'\xd0\xa0', b'\xd0\x95', b'\xd0\x9d', b'\xd0\x94',
       b' ', b'\xd0\x92', b'\xd0\x92', b'\xd0\x95', b'\xd0\xa0'],
      dtype=object)
 array([b';', b'\xd0\x98', b'\xd0\xb7', b'\xd0\xbc', b'\xd0\xb5',
       b'\xd0\xbd', b'\xd0\xb5', b'\xd0\xbd', b'\xd0\xb8'], dtype=object)]
tf.Tensor(
[b'\xd0\xa2\xd0\xa0\xd0\x95\xd0\x9d\xd0\x94 \xd0\x92\xd0\x92\xd0\x95\xd0\xa0'
 b';\xd0\x98\xd0\xb7\xd0\xbc\xd0\xb5\xd0\xbd\xd0\xb5\xd0\xbd\xd0\xb8'], shape=(2,), dtype=string)


C:\Users\Ilya\anaconda3\envs\Pandora\lib\site-packages\tensorflow\python\ops\ragged\ragged_tensor.py:1950: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(rows)


In [6]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [14]:
# Создаем набор данных полный

all_ids = ids_from_chars(tf.strings.unicode_split(an_text, 'UTF-8'))
print(all_ids)

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(5):
    print(chars_from_ids(ids).numpy().decode('utf-8'))
    
# Разбиваем на батчи по 50 символов
seq_length = 50
examples_per_epoch = len(an_text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

# Вот функция, которая принимает последовательность как ввод, дублирует и сдвигает ее, 
# чтобы выровнять ввод и метку для каждого временного шага
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

print(split_input_target(list("Tensorflow")))

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(2):
    print("Input :", text_from_ids(input_example).numpy().decode('utf-8'))
    print("Target:", text_from_ids(target_example).numpy().decode('utf-8'))

# Финализируем датасет
# Batch size
BATCH_SIZE = 32
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

print(dataset)

tf.Tensor([45 43 36 ... 65 78  8], shape=(2932,), dtype=int64)
Т
Р
Е
Н
Д
tf.Tensor(
[b'\xd0\xa2' b'\xd0\xa0' b'\xd0\x95' b'\xd0\x9d' b'\xd0\x94' b' '
 b'\xd0\x92' b'\xd0\x92' b'\xd0\x95' b'\xd0\xa0' b'\xd0\xa5' b';'
 b'\xd0\x98' b'\xd0\xb7' b'\xd0\xbc' b'\xd0\xb5' b'\xd0\xbd' b'\xd0\xb5'
 b'\xd0\xbd' b'\xd0\xb8' b'\xd0\xb5' b' ' b'\xd1\x86' b'\xd0\xb5'
 b'\xd0\xbd' b'\xd1\x8b' b' ' b'\xd0\xb7' b'\xd0\xb0' b' ' b'\xd0\xbf'
 b'\xd0\xbe' b'\xd1\x81' b'\xd0\xbb' b'\xd0\xb5' b'\xd0\xb4' b'\xd0\xbd'
 b'\xd0\xb8' b'\xd0\xb5' b' ' b'\xd0\xbf' b'\xd0\xbe' b'\xd0\xbb'
 b'\xd0\xb3' b'\xd0\xbe' b'\xd0\xb4' b'\xd0\xb0' b' ' b'\xd1\x81'
 b'\xd0\xbe' b'\xd1\x81'], shape=(51,), dtype=string)
(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'], ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])
Input : ТРЕНД ВВЕРХ;Изменение цены за последние полгода со
Target: РЕНД ВВЕРХ;Изменение цены за последние полгода сос
Input : тавляет 26.39%, а за последний месяц 11.72%. Напра
Target: авляет 26.39%, а за последний месяц

In [8]:
# Length of the vocabulary in chars
an_vocab_size = len(an_vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [59]:
# Создаем модель
class MyAnModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
    self.lstm = tf.keras.layers.LSTM(4, return_sequences=True, 
                                   return_state=True),
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else: 
      return x

In [61]:
model = MyAnModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

for input_example_batch, target_example_batch in dataset.take(2):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(32, 50, 83) # (batch_size, sequence_length, vocab_size)


In [63]:
print(model.summary())

Model: "my_an_model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      multiple                  21248     
_________________________________________________________________
gru_8 (GRU)                  multiple                  3938304   
_________________________________________________________________


ValueError: You tried to call `count_params` on lstm_2, but the layer isn't built. You can build it manually via: `lstm_2.build(batch_input_shape)`.

In [60]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print(sampled_indices)

print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy().decode('utf-8'))

[26 39 52 18 77 41 14 59 10 79 77 44 51 44 41 38  8 11 75 64 67 63  5 50
 37 10 64 28 77 33 60 26 56  3 80 26  5 55 78  0 65 57 68 11 81 59 12  5
 69 38]
Input:
 b'\xd0\x9d\xd0\x98\xd0\x97;\xd0\x98\xd0\xb7\xd0\xbc\xd0\xb5\xd0\xbd\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5 \xd1\x86\xd0\xb5\xd0\xbd\xd1\x8b \xd0\xb7\xd0\xb0 \xd0\xbf\xd0\xbe\xd1\x81\xd0\xbb\xd0\xb5\xd0\xb4\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xbf\xd0\xbe\xd0\xbb\xd0\xb3\xd0\xbe\xd0\xb4\xd0\xb0 \xd1\x81\xd0\xbe\xd1\x81\xd1\x82\xd0\xb0\xd0\xb2\xd0\xbb\xd1\x8f\xd0\xb5\xd1\x82'

Next Char Predictions:
эa%гынер2юз3%сИ


In [62]:
# Обучение
# Прикрепите оптимизатор и функцию потерь
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)
print(tf.exp(mean_loss).numpy())

# Настройте процедуру обучения с tf.keras.Model.compile метода tf.keras.Model.compile . 
# Используйте tf.keras.optimizers.Adam с аргументами по умолчанию и функцией потерь.
model.compile(optimizer='adam', loss=loss)

# Используйте tf.keras.callbacks.ModelCheckpoint чтобы убедиться, что контрольные точки сохраняются во время обучения:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Провести обучение
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Prediction shape:  (32, 50, 83)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.419919
83.089554
Epoch 1/100
1/1 [==============================] - 1s 551ms/step - loss: 4.4198
Epoch 2/100
1/1 [==============================] - 0s 411ms/step - loss: 4.3853
Epoch 3/100
1/1 [==============================] - 0s 394ms/step - loss: 4.3344
Epoch 4/100
1/1 [==============================] - 0s 481ms/step - loss: 4.1986
Epoch 5/100
1/1 [==============================] - 0s 426ms/step - loss: 3.9147
Epoch 6/100
1/1 [==============================] - 0s 485ms/step - loss: 4.0440
Epoch 7/100
1/1 [==============================] - 1s 522ms/step - loss: 4.1301
Epoch 8/100
1/1 [==============================] - 0s 458ms/step - loss: 4.1337
Epoch 9/100
1/1 [==============================] - 0s 459ms/step - loss: 4.1150
Epoch 10/100
1/1 [==============================] - 0s 452ms/step - loss: 4.0963
Epoch 11/100
1/1 [==============================] - 0s 432ms/step - loss: 4.0476
Epo

1/1 [==============================] - 0s 445ms/step - loss: 1.2161


In [39]:
# Создать текст

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [64]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()
states = None
next_char = tf.constant(['СИГНАЛ;'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

СИГНАЛ;Лина поворить о икатором э дваноследне пикутстсяветь горио, сродначное, иринах.
aД9 В1Р1ИИИИ 

________________________________________________________________________________

Run time: 0.8305926322937012
